In [1]:
from applets.core import get_connection, SecureAppletBase
from applets.util import secp256k1

class SingleUseKey(SecureAppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CD01", connection)

    def generate_fresh_key(self):
        return self.secure_request(b'\x20\x00')
        
    def get_random_pubkey(self):
        return self.secure_request(b'\x20\x01')

    def sign(self, msg:bytes):
        if len(msg)!=32:
            raise ValueError("Message hash should be 32 bytes long")
        return self.secure_request(b'\x20\x02'+msg)

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


In [4]:
app = SingleUseKey(connection)
app.select()
app.establish_secure_channel()
app.is_secure_channel_open

True

In [5]:
# generate a new random key
pub = app.generate_fresh_key()
pub.hex()

'03a34f1a01002ebda6c8d84cbe75299041bfa3bfca832932df6538e81a143d443a'

In [6]:
# you can get this pubkey if you didn't sign using this key
app.get_random_pubkey() == pub

True

In [7]:
# sign message - only one message can be signed, then new key is generated
msg = b'5'*32
sig = app.sign(msg)
sig.hex()

'304402202a7a34e2b8ced403f8ce341c51e379359e781eeeacab0c0342db1203f1787b4802204355d8e899b20f444d4599465d2fe13c0185411faf5e4c0779d22a8baf2696'

In [8]:
from io import BytesIO
from binascii import hexlify
def check_sig(signature_bin):
    s = BytesIO(signature_bin)
    compound = s.read(1)[0]
    if compound != 0x30:
        raise RuntimeError("Bad Signature")
    length = s.read(1)[0]
    if length + 2 != len(signature_bin):
        raise RuntimeError("Bad Signature Length")
    marker = s.read(1)[0]
    if marker != 0x02:
        raise RuntimeError("Bad Signature")
    rlength = s.read(1)[0]
    r = int(hexlify(s.read(rlength)), 16)
    marker = s.read(1)[0]
    if marker != 0x02:
        raise RuntimeError("Bad Signature")
    slength = s.read(1)[0]
    s = int(hexlify(s.read(slength)), 16)
    if len(signature_bin) != 6 + rlength + slength:
        raise RuntimeError("Signature too long")
    print("OK")
check_sig(sig)

RuntimeError: Bad Signature Length

In [9]:
# verify it is valid:
sig_raw = secp256k1.ecdsa_signature_parse_der(sig)
pub_raw = secp256k1.ec_pubkey_parse(pub)
secp256k1.ecdsa_verify(sig_raw, msg, pub_raw)

ValueError: Failed parsing der signature

In [ ]:
sig=bytes.fromhex("3045022100835c61cbbb91ae7a8f6b728feadc53bd9b92b8710d419e67040d579c94e27a022021e3a9a45353748144ad65de05e836b540b06440e49a98546975f6987319f409")

In [ ]:
connection.disconnect()